Summative Assignment
- Lecture : Risk and Portfolio Management
- Name : Doheun Kiel 길도흔
- student ID : 2021313121

In [477]:
# dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import qpsolvers

### Data Source


Monthly returns of the 30 largest stocks in the US.
File name: crsp.csv (uploaded on iCapmus) Sample period: 2000.01 - 2022.12 (monthly data)
Stocks: 30 largest stocks in the US as of 2022.12.31. Columns
- permno: Unique stock id
- ret: Monthly return
- prc: Price
- shrout: Shares outstanding
- Assume that the risk-free rate is 0.

In [478]:
df = pd.read_csv("./crsp-2.csv")
df["date"] = pd.to_datetime(df["date"]) # Convert "date" column into datetime data type
df

,Unnamed: 0,date,permno,ret,prc,shrout
0,2873115,2000-01-31,10104,-0.108477,49.953125,2847344.0
1,2883897,2000-02-29,10104,0.486393,74.250000,2838409.0
2,2891757,2000-03-31,10104,0.051347,78.062500,2838409.0
3,2897954,2000-04-28,10104,0.024019,79.937500,2838409.0
4,2911756,2000-05-31,10104,-0.100860,71.875000,2807572.0
...,...,...,...,...,...,...
8275,4874507,2022-08-31,92655,-0.042427,519.330017,935383.0
8276,4891888,2022-09-30,92655,-0.024339,505.040009,935383.0
8277,4897944,2022-10-31,92655,0.099220,555.150024,935383.0
8278,4908957,2022-11-30,92655,-0.013312,547.760010,934349.0


# Parameter Estimation and portfolio rebalancing

In [479]:
class StockData:
  """Class to generate panel data using each columns; ret, prc, shrout

  Args:
    data: DataFrame (<- crsp-2.csv)

  Attributes:
    data: Dataframe of initial data
    ret : Dataframe of monthly return data (panel)
    prc : Dataframe of price data (panel)
    shrout : Dataframe of share outstanding data (panel)
    mktcap : Datafroma of market capitalization data (panel)
    date_index : index of datetime
  """
  def __init__(self, data):
    self.data = data
    self.ret = pd.pivot(self.data, values="ret", index="date", columns="permno")
    self.prc = pd.pivot(self.data, values="prc", index="date", columns="permno")
    self.shrout = pd.pivot(self.data, values="shrout", index="date", columns="permno")
    self.mktcap = self.ret * self.shrout
    self.date_index = self.get_datetime_index()

  def get_datetime_index(self):
    """Get datetime index list used in converting index into datetime

    Returns:
      list of datetime index
    """
    return self.ret.index.copy().tolist()
  
  def get_ret_til(self, when : int):
    """Get returns until the input time.

    Args:
      when: int of the index which indictaes time.

    Returns:
      return dataframe until the input time.
    """
    return self.ret.loc[:self.date_index[when]]

  def gen_weight_zeros(self):
    """Generate dataframe filled with zeros which has same size of return

    Returns:
      weight dataframe filled with zeros which has same size of self.ret
    """
    return pd.DataFrame(np.zeros_like(self.ret), index=self.ret.index, columns=self.ret.columns)

  def gen_return_zeros(self):
    """Generate dataframe filled with zeros which has same size of return

    Returns:
      portfolio return dataframe filled with zeros which has same size of self.ret
    """
    return pd.DataFrame(np.zeros(len(self.ret)), index=self.ret.index, columns=["pf_value"])
  
  def rebalance(self, portfolio_function, start : str, transaction_cost : float = 0):
    """Rebalance Portfolio by expanding window

    Args:
      portfolio_function: portfolio function which returns portfolio weight.
      start: start timestamp of the window. str : datetime
      transaction_cost: transaction cost. float; base is 0 for no cost

    Returns:
      pf_ret: dataframe of portfolio returns for backtesting period
    """
    start_condition = self.ret.index > pd.to_datetime(start)
    df_weight = self.gen_weight_zeros()
    pf_ret = self.gen_return_zeros()

    for cnt,idx in enumerate(self.ret.index):
      if cnt == 0:
          continue
      
      elif idx < pd.to_datetime(start):
        continue

      else:
        target_weight = portfolio_function(cnt-1)
        current_weight = (1+self.ret.loc[self.date_index[cnt-1]]) * df_weight.loc[self.date_index[cnt-1]]
        current_weight = current_weight / current_weight.sum()
        weight_difference = np.abs(target_weight - current_weight)

        df_weight.loc[self.date_index[cnt]] = target_weight
        pf_ret.loc[self.date_index[cnt],"pf_value"]  = (self.ret.loc[self.date_index[cnt]] * df_weight.loc[self.date_index[cnt]]).sum() 
        pf_ret.loc[self.date_index[cnt],"pf_value"] -= (weight_difference.sum() * transaction_cost)
        
    return pf_ret[start_condition]

# Portfolios

- VW: value-weight portfolio
- EW: equal-weight portfolio
- MVO: mean-variance portfolio (tangent portfolio) without any constraints
- MVO+: mean-variance portfolio (tangent portfolio) with short-sale constraints
- MinVar: minimum-variance portfolio.
- Robust: Robust optimization: Same as MVO but allowing uncertainty in mean. Use a box uncertainty whose size is the same as the standard error of the mean return.

In [480]:
class Portfolio(StockData):
  """Class for each portfolio weight according to window sizes

  Type of portfolios
  (1) value_weight: value-weight portfolio
  (2) equal_weight: equal-weight portfolio 
  (3) mean_variance: mean-variance tangent portfolio
  (4) mean_variance_short_constraint: mean-variance tangent portfolio with short-sale constraints
  (5) MinVar: minimum-variance portfolio
  (6) Robust: mean-variance portfolio with box uncertainty in mean

  Args:
    data: DataFrame : crsp-2.csv

  Attributes:
  data: Dataframe of initial data
  date_index : index of datetime
  ret: Dataframe of return panel data
  prc: Dataframe of price panel data
  shrout : Dataframe of shrout panel data
  mktcap : Datafroma of market capitalization data
  """
  def __init__(self, data):
    super().__init__(data)
    
  def value_weight(self, when :int):
    """value-weight portfolio
    Weight is proportional to the size (= prc x shrout)

    Args:
      when: the last timestamp of the window

    Return:
      dataframe of value-weight
    """
    return self.mktcap.loc[self.date_index[when]] / self.mktcap.loc[self.date_index[when]].sum()

  def equal_weight(self, when : int):
    """equal-weight portfolio

    Args:
      when: the last timestamp of the window

    Return:
      dataframe of equal-weight
    """
    return np.ones_like(self.ret.loc[self.date_index[when]]) / self.ret.shape[1]
  
  def mean_variance(self, when : int):
    """mean-variance (tangent) portfolio

    Args:
      when: the last timestamp of the window

    Return:
      dataframe of weight of mean-variance portfolio
    """
    ret_window = self.get_ret_til(when)
    returns = ret_window.mean(axis=0).values
    covariance = ret_window.cov().values

    e = np.ones_like(returns)

    # Objective function
    def fobj(w, mu, C): return -(w @ mu) /  np.sqrt(w.T @ C @ w)

    # Budget constraint (equality)
    def fcon_budget(w): return w.sum() - 1 # = 0

    cons = [dict(type='eq', fun=fcon_budget)]

    w0 = e / len(e) # Initial guess
    
    res = sp.optimize.minimize(fobj, w0, args=(returns, covariance),  constraints = cons, options={'maxiter':5000})

    weight = res.x

    return weight
  
  def mean_variance_short_constraint(self, when : int):
    """mean-variance (tangent) portfolio with short-sale constraint

    Args:
      when: the last timestamp of the window

    Return:
      dataframe of weight of mean-variance portfolio with short-sale constraint weight
    """
    ret_window = self.get_ret_til(when)
    returns = ret_window.mean(axis=0).values
    covariance = ret_window.cov().values

    e = np.ones_like(returns)

    # Objective function
    def fobj(w, mu, C): return -(w @ mu) /  np.sqrt(w.T @ C @ w)

    # Budget constraint (equality)
    def fcon_budget(w): return w.sum() - 1 # = 0

    cons = [dict(type='eq', fun=fcon_budget)]

    w0 = e / len(e) # Initial guess

    # Bounds
    bounds = sp.optimize.Bounds(0 * e)
    
    res = sp.optimize.minimize(fobj, w0, args=(returns, covariance),  constraints = cons, bounds=bounds, options={'maxiter':5000})

    weight = res.x

    return weight
  
  def min_var(self, when : int):
    """minimun-variance portfolio

    Args:
      when: the last timestamp of the window

    Return:
      dataframe of weight of minimum-variance portfolio
    """
    ret_window = self.get_ret_til(when)
    returns = ret_window.mean(axis=0).values
    covariance = ret_window.cov().values

    e = np.ones_like(returns)
    weight = np.linalg.solve(covariance, e)
    weight = weight / np.sum(weight)

    return weight
  
  def robust_optimization(self, when : int):
    """mean-variance (tangent) portfolio with box uncertainty in mean

    Args:
      when: the last timestamp of the window

    Return:
      dataframe of weight of minimum-variance portfolio
    """
    ret_window = self.get_ret_til(when)
    returns = ret_window.mean(axis=0).values
    covariance = ret_window.cov().values

    e = np.ones_like(returns)

    # Objective function - Sharpe Ratio Maximisation with box uncertainty
    def fobj(w, mu, C): return -(w @ mu - np.abs(w) @ np.std(ret_window, axis=0)) /  np.sqrt(w.T @ C @ w)

    # Budget constraint (equality)
    def fcon_budget(w): return w.sum() - 1 # = 0

    cons = [dict(type='eq', fun=fcon_budget)]
    
    w0 = e / len(e) # Initial guess
    
    res = sp.optimize.minimize(fobj, w0, args=(returns, covariance),  constraints = cons, options={'maxiter':5000})

    weight = res.x
    
    return weight

# Portfolio Evaluation

In [481]:
pf = Portfolio(data=df)

In [482]:
def evaluate(*Returns: pd.DataFrame):
    """Evaluate the portfolios with each monthly returns

    Args:
      *Returns: one or more than one portfolio returns with same window

    Returns:
      (i) evaluation: return evaluation dictionary if len(Returns) == 1
        dictonary with keys:
          1) annualized mean return,
          2) annualized standard deviation,
          3) annualized sharpe ratio,
          4) cumulative return,
          5) maximum drawdown

      (ii) result: return result array consisted with more than one evaluation
    """
    result = []

    for pf_ret in Returns:
      pf_ret = np.squeeze(pf_ret) # eliminate redundunt array with single scalar
      annualized_mean_return = pf_ret.mean() * 12
      annualized_std = pf_ret.std() * np.sqrt(12)
      annualized_sharpe_ratio = annualized_mean_return / annualized_std
      cumulative_return = (1 + pf_ret).cumprod()
      drawdown = (cumulative_return - cumulative_return.cummax()) / cumulative_return
      max_drawdown = drawdown.min()

      evaluation = {
        "Mean": annualized_mean_return,
        "Std": annualized_std,
        "Sharpe": annualized_sharpe_ratio,
        "CumRet": cumulative_return.iloc[-1],
        "MDD": max_drawdown
      }

      if len(Returns) == 1: return evaluation

      else: result.append(evaluation)

    return result

### Portfolio evaluation with no transaction cost

In [483]:
VW = pf.rebalance(pf.value_weight, "2020", transaction_cost=0)
EW = pf.rebalance(pf.equal_weight, "2020", transaction_cost=0)
MVO = pf.rebalance(pf.mean_variance, "2020", transaction_cost=0)
MVO_ = pf.rebalance(pf.mean_variance_short_constraint, "2020", transaction_cost=0)
MinVar = pf.rebalance(pf.min_var, "2020", transaction_cost=0)
Robust = pf.rebalance(pf.robust_optimization, "2020", transaction_cost=0)

In [484]:
performance = pd.DataFrame(evaluate(VW, EW, MVO, MVO_, MinVar, Robust)).T
performance.columns = ["VW", "EW", "MVO", "MVO+", "MinVar", "Robust"]
performance

,VW,EW,MVO,MVO+,MinVar,Robust
Mean,0.198371,0.162721,0.540392,0.184906,0.199074,0.436772
Std,0.491405,0.182044,0.234067,0.188330,0.169853,0.510606
Sharpe,0.403680,0.893854,2.308710,0.981822,1.172035,0.855400
CumRet,1.347255,1.549169,4.536679,1.648119,1.735773,2.493006
MDD,-0.459526,-0.184363,-0.105779,-0.188198,-0.166658,-1.688890


### Portfolio evaluation with transaction cost
transaction cost = 20 basis point

In [485]:
VW = pf.rebalance(pf.value_weight, "2020", transaction_cost=0.002)
EW = pf.rebalance(pf.equal_weight, "2020", transaction_cost=0.002)
MVO = pf.rebalance(pf.mean_variance, "2020", transaction_cost=0.002)
MVO_ = pf.rebalance(pf.mean_variance_short_constraint, "2020", transaction_cost=0.002)
MinVar = pf.rebalance(pf.min_var, "2020", transaction_cost=0.002)
Robust = pf.rebalance(pf.robust_optimization, "2020", transaction_cost=0.002)

In [486]:
performance_tc = pd.DataFrame(evaluate(VW, EW, MVO, MVO_, MinVar, Robust)).T
performance_tc.columns = ["VW", "EW", "MVO", "MVO+", "MinVar", "Robust"]
performance_tc

,VW,EW,MVO,MVO+,MinVar,Robust
Mean,0.082531,0.161638,0.522602,0.183702,0.194927,0.436765
Std,0.458348,0.182019,0.233399,0.188282,0.169746,0.510606
Sharpe,0.180062,0.888025,2.239094,0.975678,1.148347,0.855387
CumRet,0.978051,1.544217,4.311552,1.642292,1.714701,2.492954
MDD,-0.519064,-0.185494,-0.106931,-0.189564,-0.167337,-1.688917


# Analysis

### 1. Compare the performance of the models. In doing so, you may compare the realized performance with the expected performance.

- MVO has the highest mean return, shrape ratio and culmulative return.
- MinVar has the lowest standard deviation among the models.
- VW has the highest maximum drawdown among the models.

### 2. Does an optimal portfolio outperform the market?

- Since the value-weight portfolio mimics the market index, so we can regard VW as the market. 
- Then, optimal portfolio (MVO) outperforms the market.
- Sharpe ratio of MVO is pretty higher than value-weight portfolio.

### 3. Does adding the short-sale constraint improves or worsen the performance and why?

- Short-sale constraint reduced the standard deviation of portfolio, it becomes less risky portfolio.
- However, sharpe ratio, the risk-adjusted performence measure, is much higher in a portfolio without the short-sale constraint.
- The short-sale constraint worsen the performance because it limits the flexibilty of the portfolio. 
- It may lead to a less efficient allocation of capital, especially if there are mispricings in the market that cannot be fully exploited due to the constraint.

### 4. Discuss the impact of transaction costs.

In [487]:
performance_tc - performance

,VW,EW,MVO,MVO+,MinVar,Robust
Mean,-0.115839,-0.001083,-0.017790,-0.001204,-0.004147,-6.852821e-06
Std,-0.033057,-0.000025,-0.000668,-0.000048,-0.000107,1.449554e-07
Sharpe,-0.223618,-0.005829,-0.069615,-0.006144,-0.023687,-1.366380e-05
CumRet,-0.369204,-0.004952,-0.225127,-0.005827,-0.021072,-5.174423e-05
MDD,-0.059538,-0.001131,-0.001152,-0.001366,-0.000678,-2.699360e-05


- Mean return and culmulative return are decreased due to the impact of transaction costs, they directly affect returns.
- But in terms of standard deviation, they become less risky portfolio actually because the returns from weight change are sometimes not bigger than the transaction cost from weight change.
- Sharpe ratio also decreased since the change in mean return is bigger than the change of standard deviation.
